In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.externals import joblib
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
import catboost as cb
import gc

%matplotlib inline

In [2]:
plt.rc('figure',dpi=100);

In [3]:
avisos = pd.read_csv("../../Data/fiuba_entrenamiento/gian/avisos.csv")
postulaciones = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulaciones.csv")
postulantes = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulantes.csv")

postulantes['sexo']=postulantes['sexo'].astype('category')
postulantes['Doctorado']=postulantes['Doctorado'].astype('category')
postulantes['Master']=postulantes['Master'].astype('category')
postulantes['Otro']=postulantes['Otro'].astype('category')
postulantes['Posgrado']=postulantes['Posgrado'].astype('category')
postulantes['Secundario']=postulantes['Secundario'].astype('category')
postulantes['Terciario/Técnico']=postulantes['Terciario/Técnico'].astype('category')
postulantes['Universitario']=postulantes['Universitario'].astype('category')

avisos['nombre_zona']=avisos['nombre_zona'].astype('category')
avisos['tipo_de_trabajo']=avisos['tipo_de_trabajo'].astype('category')
avisos['nivel_laboral']=avisos['nivel_laboral'].astype('category')
avisos['nombre_area']=avisos['nombre_area'].astype('category')

postulaciones['sepostulo']=1
postulaciones.drop(['fechapostulacion','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)

In [4]:
vistas = pd.read_csv("../../Data/fiuba_entrenamiento/gian/vistas.csv")
vistas.head()

,idAviso,timestamp,idpostulante,dia,mes,semana,diadelasemana,hora
0,1111780242,2018-02-23 18:38:13.187,YjVJQ6Z,23,2,8,Viernes,18
1,1112263876,2018-02-23 18:38:14.296,BmVpYoR,23,2,8,Viernes,18
2,1112327963,2018-02-23 18:38:14.329,wVkBzZd,23,2,8,Viernes,18
3,1112318643,2018-02-23 18:38:17.921,OqmP9pv,23,2,8,Viernes,18
4,1111903673,2018-02-23 18:38:18.973,DrpbXDP,23,2,8,Viernes,18


In [5]:
vistas.drop(['timestamp','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)
vistas['cantidad']=0
vistas2=vistas.groupby(['idAviso','idpostulante'],as_index=False).agg({'cantidad':'count'})
vistas2=vistas2.rename(columns={'idAviso': 'idaviso'})
vistas2.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,2
1,48375,RwVdKR,1
2,169730,1KjXB,2
3,169730,2AKzxa,2
4,169730,6LJ64,1


In [6]:
vistas=vistas.groupby(['idpostulante'],as_index=False).agg({'cantidad':'count'});
vistas=vistas.rename(columns={'idAviso': 'idaviso'})
vistas.head()

,idpostulante,cantidad
0,0002q,15
1,0005E,58
2,000R8,14
3,001XE,26
4,003k9,32


In [7]:
vistas2=pd.merge(vistas2,vistas,on='idpostulante',how='left')

In [8]:
vistas2.head()

,idaviso,idpostulante,cantidad_x,cantidad_y
0,18,BolNL,2,28
1,48375,RwVdKR,1,15
2,169730,1KjXB,2,99
3,169730,2AKzxa,2,2
4,169730,6LJ64,1,23


In [9]:
vistas2['cantidad']=vistas2['cantidad_x']/vistas2['cantidad_y']
vistas2.drop(['cantidad_x','cantidad_y'],axis=1,inplace=True)
vistas=vistas2
vistas2.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,0.071429
1,48375,RwVdKR,0.066667
2,169730,1KjXB,0.020202
3,169730,2AKzxa,1.000000
4,169730,6LJ64,0.043478


In [10]:
postulaciones2= pd.read_csv("../../Data/fiuba_entrenamiento/gian/no-postulaciones/7.csv")
postulaciones2=postulaciones2.append(pd.read_csv("../../Data/fiuba_entrenamiento/gian/no-postulaciones/8.csv").sample(len(postulaciones2)//2).reset_index(drop=True))
print(len(postulaciones2))
postulaciones2

9906801


,idaviso,idpostulante,sepostulo
0,1112389495,5NbOjG,0
1,1112024109,1QP841b,0
2,1112490884,96XQ6N9,0
3,1111474259,qe2QJj1,0
4,1112211292,KBzbBJm,0
5,1112328485,3NzZv26,0
6,1112474766,akDjWwE,0
7,1112312385,aZpBJW,0
8,1112285628,akDWj55,0
9,1112322843,GNd0KNN,0


In [11]:
postulaciones.head()

,idaviso,idpostulante,sepostulo
0,1112257047,NM5M,1
1,1111920714,NM5M,1
2,1112346945,NM5M,1
3,1112345547,NM5M,1
4,1112237522,5awk,1


In [12]:
postulaciones=postulaciones.append(postulaciones2, ignore_index=True)
print(len(postulaciones))
postulaciones2=0
gc.collect()
postulaciones.drop_duplicates(['idaviso','idpostulante'],keep='first',inplace=True)
print(len(postulaciones))

16511335
16501957


In [13]:
postulaciones=pd.merge(postulaciones,postulantes,on='idpostulante')
postulaciones=pd.merge(postulaciones,avisos,on='idaviso')
postulaciones=pd.merge(postulaciones,vistas,on=['idaviso','idpostulante'],how='left')
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad
10150544,1112280836,16K4wN,0,MASC,52.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,False,NaN
943123,1112268571,owjaPAr,1,FEM,26.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,NaN
959132,1112266388,6rQozKM,0,FEM,24.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,NaN
14797086,1112473283,QNL1KVq,0,MASC,22.0,-,-,Graduado,-,Graduado,...,False,True,False,False,False,False,False,False,False,NaN
3006144,1112369238,Yj5dK5w,1,FEM,25.0,-,-,-,-,Graduado,...,False,True,False,False,False,False,False,False,False,NaN
3645992,1112285974,aZBpoq,1,FEM,31.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,NaN
548764,1112318470,YjlZ4qX,1,FEM,41.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,True,False,NaN
3126225,1112247061,EKXG1N,1,FEM,33.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,NaN
1772410,1112416189,EzeO0K4,1,FEM,20.0,-,-,-,-,-,...,False,False,False,False,True,True,False,False,False,0.006369
5677514,1112355065,PmLrJdY,1,MASC,29.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,NaN


In [14]:
postulaciones.cantidad=postulaciones.cantidad.fillna(0)
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad
7289797,1112428743,ekOP2o4,0,NO_DECLARA,200.0,-,-,-,-,-,...,False,False,True,False,False,False,False,True,False,0.0
6536333,1112281392,Yjox04w,1,FEM,28.0,-,-,-,-,Graduado,...,False,True,False,False,False,False,False,False,False,0.0
12423671,1112216935,11mDVQ,0,MASC,37.0,-,-,-,-,Graduado,...,False,False,True,False,False,False,False,False,False,0.0
12027513,1112495493,5mPxVXk,0,MASC,35.0,-,-,-,-,Graduado,...,False,True,False,False,False,False,False,False,False,0.0
3529031,1112323659,eLdPOo,0,FEM,33.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,False,0.0
6039100,1112317131,Oqewq4v,0,MASC,42.0,-,En Curso,Graduado,Graduado,Graduado,...,False,True,False,True,False,False,False,False,False,0.0
11424888,1112381227,pzdK18W,0,MASC,21.0,-,-,-,-,Graduado,...,False,False,True,False,False,False,False,False,False,0.0
5965597,1112283590,VNx6vQN,1,MASC,33.0,-,-,-,-,-,...,False,True,False,False,False,False,False,False,False,0.0
11761264,1112456271,dYjLRlX,0,FEM,38.0,-,-,-,-,-,...,False,False,True,False,False,True,False,False,False,0.0
3003830,1112369238,8Go4eW,0,MASC,28.0,-,-,-,-,Graduado,...,False,True,False,False,False,False,False,False,False,0.0


In [15]:
train, test=train_test_split(postulaciones,test_size=0.10)
print("Train: ",len(train),"Test: ",len(test))
features=list(postulaciones.columns)
features.remove('idaviso')
features.remove('idpostulante')
features.remove('sepostulo')
features.remove('titulo')
features.remove('denominacion_empresa')

x_train=train[features]
y_train=train['sepostulo']

x_test=test[features]
y_test=test['sepostulo']
postulaciones=0
vistas2=0
train=0
test=0

Train:  14477520 Test:  1608614


In [16]:
gc.collect()

300

In [53]:
c=lgb.LGBMClassifier(learning_rate=0.01,objective='binary',num_leaves=7000,max_depth=18,n_estimators=10,colsample_bytree=1,n_jobs=-1,random_state=0,silent=False,subsample=1,sumsample_freq=1)

dt=c.fit(x_train,y_train,eval_set=[(x_test,y_test)],verbose=True)
#c = joblib.load("../../Data/fiuba_entrenamiento/gian/modelos/....pkl")

[1]	valid_0's binary_logloss: 0.688594
[2]	valid_0's binary_logloss: 0.684132
[3]	valid_0's binary_logloss: 0.679755
[4]	valid_0's binary_logloss: 0.675463
[5]	valid_0's binary_logloss: 0.671253
[6]	valid_0's binary_logloss: 0.667119
[7]	valid_0's binary_logloss: 0.663061
[8]	valid_0's binary_logloss: 0.659081
[9]	valid_0's binary_logloss: 0.655171
[10]	valid_0's binary_logloss: 0.651336


In [54]:
score=c.score(x_test,y_test)*100
print(score)

80.20643858626121


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


![title](img/grid_lightgbm.png)

In [19]:
prediccion = pd.read_csv("../../Data/fiuba_entrenamiento/test_final_100k.csv")
print(len(prediccion))
prediccion=pd.merge(prediccion,postulantes,on='idpostulante',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,avisos,on='idaviso',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,vistas,on=['idaviso','idpostulante'],how='left')
prediccion.cantidad=prediccion.cantidad.fillna(0)
prediccion.sample(10)

100000
100000
100000


,id,idaviso,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad
71237,16691,1112177862,eMGE8r,FEM,38.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,True,0.000000
41257,75359,1112464097,MVr2MGv,MASC,27.0,-,-,-,-,Graduado,...,True,False,True,False,False,False,False,True,False,0.008621
50830,58247,1112452448,82zD0W,FEM,38.0,-,-,Graduado,-,-,...,False,False,False,False,False,False,False,True,False,0.000000
87738,41925,1112422119,6J4PWl,FEM,36.0,-,-,-,-,Graduado,...,False,True,False,False,False,True,False,False,False,0.000000
36359,40287,1112418765,61YeGx,FEM,29.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.000000
60429,8642,1111787726,EXL64,NO_DECLARA,200.0,-,-,-,-,-,...,False,False,True,False,True,False,False,False,False,0.000000
95533,94978,1112486925,Nj3bGJ,MASC,48.0,-,-,-,-,Graduado,...,False,True,False,False,True,False,False,True,False,0.000000
1985,83945,1112466470,eGDJ6r,FEM,32.0,-,-,En Curso,-,Graduado,...,True,False,True,False,True,True,False,False,False,0.000000
52066,3961,1111420075,YMp401,FEM,28.0,-,-,Graduado,Graduado,Graduado,...,False,False,False,False,False,False,False,False,False,0.000000
29852,81391,1112465835,vVkWLKo,MASC,31.0,-,-,-,-,Graduado,...,False,False,True,False,False,False,False,False,False,0.000000


In [20]:
x_final=prediccion[features]
y_final=c.predict_proba(x_final)
y_final

array([[0.51242   , 0.48758   ],
       [0.5139701 , 0.4860299 ],
       [0.51166576, 0.48833424],
       ...,
       [0.51067713, 0.48932287],
       [0.50962413, 0.49037587],
       [0.52095805, 0.47904195]])

In [21]:
y_final2=[]
print(c.classes_)
print(len(y_final))
for elem in y_final:
    y_final2.append(elem[1])
print(len(y_final2))

[0 1]
100000
100000


In [22]:
prediccion['sepostulo']=y_final2

In [23]:
prediccion.head()

,id,idaviso,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad,sepostulo
0,0,739260,6M9ZQR,FEM,42.0,Graduado,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,0.0,0.487580
1,1,739260,6v1xdL,MASC,30.0,-,-,Graduado,-,-,...,False,False,False,False,False,False,False,False,0.0,0.486030
2,2,739260,ezRKm9,FEM,36.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,0.0,0.488334
3,3,758580,1Q35ej,MASC,68.0,-,-,-,Graduado,Graduado,...,False,False,False,False,False,False,False,False,0.0,0.482110
4,4,758580,EAN4J6,FEM,32.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,0.0,0.481983


In [24]:
prediccion.drop(features,axis=1,inplace=True)
prediccion.drop(['idaviso','idpostulante','titulo','denominacion_empresa'],axis=1,inplace=True)
prediccion.head()

,id,sepostulo
0,0,0.487580
1,1,0.486030
2,2,0.488334
3,3,0.482110
4,4,0.481983


In [25]:
prediccion.to_csv("../../Data/fiuba_entrenamiento/gian/predicciones/sample.csv",index=False)

In [26]:
prediccion.sepostulo.mean()

0.5005465269721076